In [1]:
import ctypes
import os
import sys
import tempfile

In [2]:
os.path.abspath(os.path.curdir)

'C:\\Python\\Projects\\lmt\\canapelib'

In [3]:
working_directory=os.path.abspath(tempfile.mkdtemp(suffix="_canape",prefix="tmp_",dir=os.curdir))

In [4]:
api_path=r"C:\Program Files (x86)\Vector CANape 14\CANapeAPI"

- http://vector.com/portal/medien/cmc/application_notes/AN-AMC-1-103_CANape_ASAM_MCD3_Interface.pdf

Access to the ASAP3 interface is possible via three different methods:
1. CANapAPI.DLL: The ASAP3 functions are realized via DLL functions locally
2. CANaptcp.DLL: Remote access to CANape via client application using DLL functions
3. COM interface: This interface enlarges the CANapeAPI functions. Other programs have access to ECUs
via the Microsoft COM interface (Vector Application Note AN67-15)

### The CANapeAPI.DLL provides the following functionalities:
1. Exchange of data between CANape and other external applications
2. Creation of an mdf (Measurement Data Format) file to log Measurement Data
3. Automation of calibration sequences with customer-specific user interface
4. Execute script files and services

### The CANaptcp.DLL provides the following functionalities:
1. All functions included in the CANapAPI.DLL
2. Remote TCP/IP access instead of local direct access

# Method 0: COM

Connect to CANape using the COM interface. 

Pros:
- COM interface.

Cons:
- COM interface. 
- Stuck with a single version of CANape (which ever one has its DLL registered).
- To register a new version requires Admin access.

In [ ]:
import win32com.client
hdl = win32com.client.Dispatch('CANAPE.Application')

In [ ]:
hdl.Open(working_directory,0)

In [ ]:
hdl.Quit()

In [ ]:
hdl.Open1(working_directory,1,1000,True)

In [ ]:
hdl.Quit()

# Method 1: CANapAPI

Connect to CANape using the API, directly.

Pros:
- 32 and 64-bit versions.
- Use any version of CANape you have installed.
- More hard core.

Cons:
- Requires use of CTypes
- ~~I had to write everything below.~~ [Magic](https://github.com/davidjamesca/ctypesgen)

In [5]:
import platform
import CANapAPI

In [6]:
if platform.architecture()[0] == '32bit':
    canape_api=ctypes.WinDLL(os.path.join(api_path,"CANapAPI.dll"))
else:
    canape_api=ctypes.WinDLL(os.path.join(api_path,"CANapAPI64.dll"))

In [7]:
ver = CANapAPI.version_t()

In [8]:
r = canape_api.Asap3GetVersion(ctypes.byref(ver))
print(r)
print(ver)

58779905


In [9]:
for field, ct in CANapAPI.version_t._fields_:
    print("{}: {}".format(field,getattr(ver,field)))

dllMainVersion: 2
dllSubVersion: 3
dllRelease: 1
osVersion: Windows95/WindowsNT
osRelease: 0


In [14]:
CANapAPI.tAsap3Hdl

CANapAPI.struct_tAsap3Hdl

In [15]:
CANapAPI.TAsap3Hdl

CANapAPI.LP_struct_tAsap3Hdl

AttributeError: 'module' object has no attribute 'None'

In [19]:
Hdl=None
responseTimeout=200000
workingDir = working_directory
fifoSize = 8192
debugMode = True

In [24]:
ctypes.c_char

ctypes.c_char

In [28]:
init=canape_api.Asap3Init
init.argtypes = (ctypes.POINTER(CANapAPI.TAsap3Hdl),
                 ctypes.c_ulong,
                 ctypes.c_char_p,
                 ctypes.c_ulong,
                 ctypes.c_bool
                )
init.restype = ctypes.c_bool

In [20]:
canape_api.Asap3Init(ctypes.byref(Hdl),
                     responseTimeout,
                     workingDir,
                     fifoSize,
                     debugMode)

TypeError: byref() argument must be a ctypes instance, not 'NoneType'

In [ ]:
canape_api.Asap3PopupDebugWindow(hdl)

In [63]:
hdl

In [61]:
canape_api.Asap3Exit(ctypes.byref(hdl))

0

# Method 2: CANaptcp

Connect to CANape using the TCP/IP API, directly.

Pros:
- 32 and 64-bit versions.
- Use any version of CANape you have installed.
- Use CANape on a remote machine.

Cons:
- Requires use of CTypes.
- CANape must be started manually on the remote machine.

In [ ]:
canape_api=ctypes.WinDLL(os.path.join(api_path,"CANapAPI.dll"))

In [ ]:
hdl = ctypes.c_uint32()

In [ ]:
m_WorkingDir=os.path.abspath(os.curdir)
m_WorkingDir

In [ ]:
canape_api =canape_api.Asap3Init5(ctypes.byref(hdl), 20000, m_WorkingDir, 1000, 1, True, True, False, False);

In [ ]:
import platform

In [ ]:
platform.architecture()